# Plotly Charts

In [14]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import dash 
from dash import dcc 
from dash import html
from dash.dependencies import Input, Output

## Loading Dataset

In [16]:
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1", dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
airline_data.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,...,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1295781,1998,2,4,2,4,1998-04-02,AS,19930,AS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1125375,2013,2,5,13,1,2013-05-13,EV,20366,EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118824,1993,3,9,25,6,1993-09-25,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634825,1994,4,11,12,6,1994-11-12,HP,19991,HP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1888125,2017,3,8,17,4,2017-08-17,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
airline_data.shape

(27000, 110)

In [20]:
data = airline_data.sample(n=27000, random_state=42)
data.shape

(27000, 110)

In [22]:
data.Month

5312      3
18357     8
6428      7
15414    10
10610     2
         ..
21575    12
5390     11
860       1
15795    11
23654     5
Name: Month, Length: 27000, dtype: int64

In [24]:
data1 = airline_data.sample(n=1000, random_state=42)

### Plot 1

In [54]:
px.scatter(data_frame=data1, title="Distance vs Departure Time", x="Distance", y="DepTime", color_discrete_sequence=["red"])

### Plot 2

In [56]:
df= data1.groupby("Month")["ArrDelay"].mean().reset_index()
px.line(data_frame=df, title="Month vs Average Flight Delay Time", x="Month", y="ArrDelay", color_discrete_sequence=["green"])

### Plot 3

In [58]:
df2= data1.groupby("DestState")["Flights"].sum().reset_index()
px.bar(data_frame=df2, x="DestState", y="Flights", title="Total number of flights to the destination state split by reporting air")

### Plot 4

In [60]:
data1["ArrDelay"] = data1["ArrDelay"].fillna(0)
px.histogram(title="Total number of flights to the destination state split by reporting air", x="ArrDelay", data_frame=data1)

### Plot 5

In [62]:
df4 = data1.groupby("Reporting_Airline")["Flights"].sum().reset_index()
px.scatter(x="Reporting_Airline", y="Flights", data_frame=df4, size="Flights", size_max=70, hover_name="Reporting_Airline")

### Plot 6

In [64]:
df5= data1["DestCityName"].value_counts().head(10).reset_index()
px.pie(values="count", names="DestCityName", title="DestCityName proportion", data_frame=df5)

### Plot 7

In [66]:
grouped_data = data1.groupby(['Month', 'DistanceGroup']).size().reset_index(name='Count')
px.sunburst(grouped_data, path=['Month', 'DistanceGroup'], values='Count', title="Distance group proportion by month")

### Plot 8

In [68]:
grouped_data1 = data1.groupby(['Month', 'DestStateName'])["Flights"].sum().reset_index()
px.sunburst(grouped_data1, path=['Month', 'DestStateName'], values='Flights', title="Flight Distribution Hierarchy")

### Plot 9

In [70]:
fig = px.pie(data, values='Flights', names='DistanceGroup', title='Distance group proportion by flights')

app = dash.Dash(__name__)
app.layout = html.Div(children=[html.H1('Airline Performance Dashboard',style={'textAlign': 'center', 'color': '#503D36', 'font-size': 30}), 
                                html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', type='number', style={'height':'40px', 'font-size': 15})], 
                                         style={'font-size': 20}),
                                html.Br(), 
                                html.Br(), 
                                html.Div(dcc.Graph(id='line-plot'))])
if __name__ == '__main__': 
    app.run()

### Plot 10

In [72]:
app = dash.Dash(__name__)

app.layout = html.Div(children=[html.H1('Airline Performance Dashboard',style={'textAlign': 'center', 'color': '#503D36', 'font-size': 30}), 
                                html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', type='number', style={'height':'30px', 'font-size': 15})], 
                                         style={'font-size': 20}),
                                html.Br(), 
                                html.Br(), 
                                html.Div(dcc.Graph(id='line-plot'))])


@app.callback(Output(component_id='line-plot', component_property='figure'), Input(component_id='input-year', component_property='value'))

# Add computation to callback function and return graph
def get_graph(entered_year): 
    # Select 2019 data 
    df = data[data['Year']==int(entered_year)] 
    # Group the data by Month and compute average over arrival delay time. 
    line_data = df.groupby('Month')['ArrDelay'].mean().reset_index() 
    fig = go.Figure(data=go.Scatter(x=line_data['Month'], y=line_data['ArrDelay'], mode='lines', marker=dict(color='green'))) 
    fig.update_layout(title='Month vs Average Flight Delay Time', xaxis_title='Month', yaxis_title='ArrDelay') 
    return fig



if __name__ == '__main__': 
    app.run()

### Plot 11

In [74]:
app = dash.Dash(__name__)

app.layout = html.Div(children=[html.H1('Total number of flights to the destination state split by reporting air',style={'textAlign': 'center', 'color': '#503D36', 'font-size': 30}), 
                                html.Div(dcc.Dropdown(options=[{'label': 'Option 1', 'value': '1'}, {'label': 'Option 2', 'value': '2'}], value='1')),
                                html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', type='number', style={'height':'50px', 'font-size': 15})], 
                                         style={'font-size': 20}),
                                html.Br(), 
                                html.Br(), 
                                html.Div(dcc.Graph(id='bar-plot'))])


@app.callback(Output(component_id='bar-plot', component_property='figure'), Input(component_id='input-year', component_property='value'))

# Add computation to callback function and return graph
def get_graph(entered_year): 
    # Select 2019 data 
    df = data[data['Year']==int(entered_year)] 
    # Group the data by Month and compute average over arrival delay time. 
    bar_data = df.groupby(['DestState', 'Reporting_Airline'])['Flights'].sum().reset_index() 
    fig = px.bar(data_frame=bar_data, x='DestState', y='Flights', hover_name="Reporting_Airline") 
    fig.update_layout(title='Flights to Destination State', xaxis_title='Destination State', yaxis_title='Flights') 
    return fig



if __name__ == '__main__': 
    app.run()

### Plot 12

In [85]:
app = dash.Dash(__name__)
app.layout = html.Div(children=[
    html.H1('Flight Delay Time Statistics', style={
        'textAlign': 'center', 'color': '#503D36', 'font-size': 30}),
    
    html.Div([
        "Input Year: ", 
        dcc.Input(id='input-year', value='2010', type='number',
                  style={'height': '35px', 'font-size': 20}),
    ], style={'font-size': 20, 'margin': '20px'}),
    
    html.Div([
        dcc.Graph(id='carrier-plot', style={'width': '100%', 'padding': '10px'}),
        dcc.Graph(id='weather-plot', style={'width': '100%', 'padding': '10px'}),
        dcc.Graph(id='nas-plot', style={'width': '100%', 'padding': '10px'}),
        dcc.Graph(id='security-plot', style={'width': '100%', 'padding': '10px'}),
        dcc.Graph(id='late-plot', style={'width': '100%', 'padding': '10px'})
    ], style={'display': 'flex', 'flexDirection': 'column', 'flexWrap': 'wrap'})
])

def compute_info(data, entered_year): 
    """
    This function takes in airline data and selected year as an input and performs computation for creating charts and plots.
    Arguments: 
    airline_data: Input airline data. 
    entered_year: Input year for which computation needs to be performed. 
    Returns: 
    Computed average dataframes for carrier delay, weather delay, NAS delay, security delay, and late aircraft delay.
    """
    df = data[data['Year']==int(entered_year)] 
    # Compute delay averages 
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index() 
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index() 
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index() 
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index() 
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index() 
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late
    
"""Callback Function
Function that returns fugures using the provided input year.
Arguments: 
entered_year: Input year provided by the user. 


Returns: 
List of figures computed using the provided helper function `compute_info`.
"""
# Callback decorator
@app.callback([Output(component_id='carrier-plot', component_property='figure'), 
               Output(component_id='weather-plot', component_property='figure'), 
               Output(component_id='nas-plot', component_property='figure'), 
               Output(component_id='security-plot', component_property='figure'), 
               Output(component_id='late-plot', component_property='figure')], 
              Input(component_id='input-year', component_property='value'))
def get_graph(entered_year): 
    # Compute required information for creating graph from the data 
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(data, entered_year) 
    # Line plot for carrier delay 
    carrier_fig = px.bar(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline') 
    # Line plot for weather delay 
    weather_fig = px.bar(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline') 
    # Line plot for nas delay 
    nas_fig = px.bar(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline') 
    # Line plot for security delay 
    sec_fig = px.scatter(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline') 
    # Line plot for late aircraft delay 
    late_fig = px.bar(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline') 
    return[carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]


if __name__ == '__main__': 
    app.run()